In [1]:
import pandas as pd
import numpy as np

from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

import warnings
# warnings.filterwarnings('ignore')

### Clean Chess Dataset

In [5]:
# https://www.kaggle.com/datasnaek/chess
chess_df = pd.read_csv('Data/games.csv')
chess_df.head()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [6]:
chess_num_cols = ['created_at', 'last_move_at', 'turns', 'white_rating', 'black_rating', 'opening_ply']
for num_col in chess_num_cols:
    chess_df[num_col] = (chess_df[num_col] - chess_df[num_col].mean()) /chess_df[num_col].std()

chess_df['winner_white'] = chess_df['winner'] == 'white'
chess_df = chess_df[['rated', 'created_at', 'last_move_at', 'turns', 'victory_status',
                     'white_rating', 'black_rating', 'opening_eco', 'opening_ply', 'winner_white']]

# max_openings = 30
# popular_opennings = chess_df['opening_eco'].value_counts()[:max_openings].index.tolist()
# def replace_opening(x):
#     if (x in popular_opennings):
#         return x
#     else:
#         return 'other'

# chess_df['opening_eco'] = chess_df['opening_eco'].apply(replace_opening)

chess_df.head()

,rated,created_at,last_move_at,turns,victory_status,white_rating,black_rating,opening_eco,opening_ply,winner_white
0,False,0.722528,0.722501,-1.413916,outoftime,-0.331779,-1.366951,D10,0.065431,True
1,True,0.719721,0.719694,-1.324552,resign,-0.942931,-1.126431,B00,-0.292076,False
2,True,0.719721,0.719694,0.015907,mate,-0.345513,-0.305227,C20,-0.649582,True
3,True,0.719020,0.718992,0.015907,mate,-0.541219,-0.463283,D02,-0.649582,True
4,True,0.716213,0.716185,1.028698,mate,-0.252810,-0.411743,C41,0.065431,True


In [7]:
chess_df_X = chess_df.drop(columns=['winner_white'])
chess_df_y = chess_df['winner_white']

In [8]:
chess_X_cat_col = ['rated', 'victory_status', 'opening_eco']
chess_X = pd.get_dummies(columns=chess_X_cat_col, data=chess_df_X)

chess_y = chess_df_y.replace({True: 1, False: 0})

In [9]:
chess_y.value_counts()

0    10057
1    10001
Name: winner_white, dtype: int64

### Clean Mushrooms Dataset

class: edible(e), poisonous(p)

cap-shape: bell(b), conical(c), convex(x), flat(f), knobbed(k), sunken(s)

cap-surface: fibrous(f), grooves(g), scaly(y), smooth(s)

cap-color: brown(n), buff(b), cinnamon(c), gray(g), green(r), pink(p), purple(u), red(e), white(w), yellow(y)

bruises: bruises(t), no(f)

odor: almond(a), anise(l), creosote(c), fishy(y), foul(f), musty(m), none(n), pungent(p), spicy(s)

gill-attachment: attached(a), descending(d), free(f), notched(n)

gill-spacing: close(c), crowded(w), distant(d)

gill-size: broad(b), narrow(n)

gill-color: black(k), brown(n), buff(b), chocolate(h), gray(g), green(r), orange(o), pink(p), purple(u), red(e), white(w), yellow(y)

stalk-shape: enlarging(e), tapering(t)

stalk-root: bulbous(b), club(c), cup(u), equal(e), rhizomorphs(z), rooted(r), missing(?)

stalk-surface-above-ring: fibrous(f), scaly(y), silky(k), smooth(s)

stalk-surface-below-ring: fibrous(f), scaly(y), silky(k), smooth(s)

stalk-color-above-ring: brown(n), buff(b), cinnamon(c), gray(g), orange(o), pink(p), red(e), white(w), yellow(y)

stalk-color-below-ring: brown(n), buff(b), cinnamon(c), gray(g), orange(o), pink(p), red(e), white(w), yellow(y)

veil-type: partial(p), universal(u)

veil-color: brown(n), orange(o), white(w), yellow(y)

ring-number: none(n), one(o), two(t)

ring-type: cobwebby(c), evanescent(e), flaring(f), large(l), none(n), pendant(p), sheathing(s), zone(z)

spore-print-color: black(k), brown(n), buff(b), chocolate(h), green(r), orange(o), purple(u), white(w), yellow(y)

population: abundant(a), clustered(c), numerous(n), scattered(s), several(v), solitary(y)

habitat: grasses(g), leaves(l), meadows(m), paths(p), urban(u), waste(w), woods(d)

In [16]:
# https://www.kaggle.com/uciml/mushroom-classification
shrooms = pd.read_csv('Data/mushrooms.csv')
shrooms.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [17]:
shrooms_df_X = shrooms.drop(columns=['class'])
shrooms_df_y = shrooms['class']

In [18]:
shrooms_X = pd.get_dummies(data=shrooms_df_X)
shrooms_y = shrooms_df_y.replace({'e': 0, 'p': 1})

In [21]:
shrooms_y.value_counts()

0    4208
1    3916
Name: class, dtype: int64

### Clean Cardio Dataset

Retrieved from the kaggle site https://www.kaggle.com/sulianova/cardiovascular-disease-dataset, this cardio dataset has 70000 samples and 12 variables, which were collected at the moment of medical examination. It contains a target variable that indicates the presence or absence of cardiovascular disease, as well as 11 features that might be associated with the presence of cardiovascular disease, such as age, gender, and blood pressure. There are 3 types of 11 input features:
- objective feature: factual information
- examination feature: results of medical examination
- subjective feature: information given by the patient

A more detailed description of 11 features are shown below:

- age: objective feature, int (days)
- height: objective feature, int (cm)
- weight: objective feature, float (kg)
- gender: objective feature, categorical code, 1: male, 2:female
- ap_hi: systolic blood pressure, examination feature, int
- ap_lo: diastolic blood pressure, examination feature, int
- cholesterol: examination feature, categorical code, 1: normal, 2: above normal, 3: well above normal
- gluc: glucose, examination feature, categorical code, 1: normal, 2: above normal, 3: well above normal
- smoke: subjective feature, binary, 0: do not smoke, 1: smoke
- alco: alcohol intake, subjective feature, binary, 0: do not drink alcohol, 1: drink alcohol
- active: physical activity, subjective feature, binary, 0: not physically active, 1: physically active

A detailed description of the target variable is shown below: 

- cardio: presence or absence of cardiovascular disease, binary, 0: disease not present, 1: disease present

For this dataset, we want use those 11 input features and apply machine learning algorithms to predict whether a person has cardiovascular disease or not.

In [2]:
# load the cardio dataset
cardio = pd.read_csv('data/cardio.csv', delimiter = ';')
cardio.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [3]:
# no missing values in cardio dataset
cardio.isnull().sum()

id             0
age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

In [4]:
# drop unnecessary column "id"
cardio = cardio.drop(columns = ['id'])
# convert age in days to age in years
cardio['age'] = cardio['age'].apply(lambda x: int(x/365))

In [5]:
# one hot encoding categorical input features stored in cate_cols
cardio_cate_cols = ['gender', 'cholesterol', 'gluc', 'smoke', 'alco', 'active']
cardio = pd.get_dummies(columns = cardio_cate_cols, data = cardio)

In [6]:
# scale numerical attributes to 0 mean 1 std
cardio_num_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
for num_col in cardio_num_cols:
    cardio[num_col] = (cardio[num_col] - cardio[num_col].mean()) / cardio[num_col].std()

In [7]:
# a look at cleaned dataset
cardio.head()

,age,height,weight,ap_hi,ap_lo,cardio,gender_1,gender_2,cholesterol_1,cholesterol_2,cholesterol_3,gluc_1,gluc_2,gluc_3,smoke_0,smoke_1,alco_0,alco_1,active_0,active_1
0,-0.419797,0.443449,-0.847867,-0.122181,-0.088238,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1
1,0.319108,-1.018161,0.749826,0.072610,-0.035180,1,1,0,0,0,1,1,0,0,1,0,1,0,0,1
2,-0.272016,0.078046,-0.708937,0.007679,-0.141296,1,1,0,0,0,1,1,0,0,1,0,1,0,1,0
3,-0.715359,0.565250,0.541431,0.137540,0.017878,1,0,1,1,0,0,1,0,0,1,0,1,0,0,1
4,-0.863140,-1.018161,-1.264657,-0.187111,-0.194354,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0


In [8]:
# 50.03% negative labels, 49.97% positive labels
cardio['cardio'].value_counts(normalize = True)

0    0.5003
1    0.4997
Name: cardio, dtype: float64

In [9]:
# split the cardio dataset into input features and labels 
cardio_X = cardio.drop(columns=['cardio']) # input features
cardio_y = cardio['cardio'] # true lables

### Clean Rain Dataset

Retrieved from the kaggle site https://www.kaggle.com/jsphyg/weather-dataset-rattle-package, this Rain in Australia dataset contains about 10 years of daily weather observations from many locations across Australia. There are 145460 samples and 23 variables in this dataset. It contains a target variable that indicates whether it rained the next day, as well as 22 features that might be associated with the target variable, such as minimum temperature, maximum temperature, rainfall of the day.

A more detailed description of 22 features are shown below:

- Date: the date of observation
- Location: the common name of the location of the weather station
- MinTemp: the minimum temperature in degrees celsius
- MaxTemp: the maximum temperature in degrees celsius
- Rainfall: the amount of rainfall recorded for the day in mm
- Evaporation: the so-called Class A pan evaporation (mm) in the 24 hours to 9am
- Sunshine: the number of hours of bright sunshine in the day
- WindGustDir: the direction of the strongest wind gust in the 24 hours to midnight
- WindGustSpeed: the speed (km/h) of the strongest wind gust in the 24 hours to midnight
- WindDir9am: direction of the wind at 9am
- WindDir3pm: direction of the wind at 3pm
- WindSpeed9am: wind speed (km/hr) averaged over 10 minutes prior to 9am
- WindSpeed3pm: wind speed (km/hr) averaged over 10 minutes prior to 3pm
- Humidity9am: humidity (percent) at 9am
- Humidity3pm: humidity (percent) at 3pm
- Pressure9am: atmospheric pressure (hpa) reduced to mean sea level at 9am
- Pressure3pm: atmospheric pressure (hpa) reduced to mean sea level at 3pm
- Cloud9am: fraction of sky obscured by cloud at 9am. This is measured in "oktas", which are a unit of eigths. It records how many eigths of the sky are obscured by cloud. A 0 measure indicates completely clear sky whilst an 8 indicates that it is completely overcast
- Cloud3pm: fraction of sky obscured by cloud (in "oktas": eighths) at 3pm. See Cload9am for a description of the values
- Temp9am: temperature (degrees C) at 9am
- Temp3pm: temperature (degrees C) at 3pm
- RainToday: whether the precipitation (mm) in the 24 hours to 9am exceeded 1mm, Yes: the precipitation exceeded 1mm, No: it did not exceed 1mm

A detailed description of the target variable is shown below: 

- RainTomorrow: whether amount of next day rain exceeded 1mm, Yes: next day precipitation exceeded 1mm, No: it did not exceed 1mm

For this dataset, we want use those 22 input features and apply machine learning algorithms to predict whether it rained the next day or not.

Data source: http://www.bom.gov.au/climate/dwo/ and http://www.bom.gov.au/climate/data.

In [10]:
# load Australian rain dataset
aus = pd.read_csv('Data/weatherAUS.csv')
aus.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [11]:
# display the number of missing values in each column
aus.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [12]:
# fill missing values in categorical columns with the mode 
aus_cate_cols = aus.dtypes.index[aus.dtypes == "object"].tolist()
for cate_col in aus_cate_cols:
    aus[cate_col] = aus[cate_col].fillna(aus[cate_col].mode()[0])

In [13]:
# fill missing values in numerical columns with the mean
aus_num_cols = aus.dtypes.index[aus.dtypes == "float64"].tolist()
for num_col in aus_num_cols:
    aus[num_col] = aus[num_col].fillna(aus[num_col].mean())
    # scale numerical attributes to 0 mean 1 std
    aus[num_col] = (aus[num_col] - aus[num_col].mean()) / aus[num_col].std()

In [14]:
# all missing values are filled
aus.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

In [15]:
# split the date of each observation into year, month, and day
splitted_date = aus['Date'].str.split('-')

# create 'Year', 'Month', 'Day' columns using splitted results of the date
aus['Year'] = splitted_date.str[0].astype(int)
aus['Month'] = splitted_date.str[1].astype(int)
aus['Day'] = splitted_date.str[2].astype(int)

# drop original 'Date' column
aus = aus.drop(columns = ['Date'])

In [16]:
# use 0 and 1 to indicate whether it rained or not
# 0: it rained, 1: it did not rain
aus['RainToday'] = aus['RainToday'].replace({'No': 0, 'Yes': 1})
aus['RainTomorrow'] = aus['RainTomorrow'].replace({'No': 0, 'Yes': 1})

In [17]:
# one hot encoding all categorical columns
cate_cols = aus.dtypes.index[aus.dtypes == "object"].tolist()
aus = pd.get_dummies(columns = cate_cols, data = aus)
aus.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,0.189446,-0.045336,-0.210071,-1.737284e-12,-1.385234e-12,0.302233,0.672219,0.612321,0.112394,-1.442960,...,0,0,0,0,0,0,0,0,1,0
1,-0.753098,0.265042,-0.281649,-1.737284e-12,-1.385234e-12,0.302233,-1.133434,0.382873,-1.319604,-1.296413,...,0,0,0,0,0,0,0,0,0,1
2,0.110901,0.349691,-0.281649,-1.737284e-12,-1.385234e-12,0.454692,0.559366,0.841768,-1.637826,-1.052167,...,0,0,0,0,0,0,0,0,0,1
3,-0.470335,0.674177,-0.281649,-1.737284e-12,-1.385234e-12,-1.222360,-0.343461,-1.108537,-1.266567,-1.736055,...,0,0,0,0,0,0,0,0,0,0
4,0.833518,1.280826,-0.162353,-1.737284e-12,-1.385234e-12,0.073544,-0.794874,0.153425,0.695801,-0.905620,...,0,1,0,0,0,0,0,0,0,0


In [18]:
# 78.0854% negative labels, 21.9146% positive labels
aus['RainTomorrow'].value_counts(normalize = True)

0    0.780854
1    0.219146
Name: RainTomorrow, dtype: float64

In [19]:
# split the rain dataset into input features and labels 
aus_X = aus.drop(columns=['RainTomorrow']) # input features
aus_y = aus['RainTomorrow'] # true lables

### Clean BnB Dataset

The dataset is from https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data?select=AB_NYC_2019.csv, and contains data from AirBnB listing and metrics in New York City, New York for the year 2019. There are 47900 unique values in this dataset, in which we will be using different features to predict whether an AirBnB pricing is expensive or not.

In [3]:
# load the AirBnb dataset
airbnb = pd.read_csv('data/AB_NYC_2019.csv')
airbnb.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [4]:
# We will be looking at these features of the AirBnB for our predictive model
airbnb = airbnb[['neighbourhood_group','latitude','longitude','room_type', 'price',
                       'minimum_nights','number_of_reviews','availability_365']]

In [5]:
# The price median will be the threshold for our expensive classifier
airbnb['price'].median()

106.0

In [6]:
# Creating a new column with prices greater than the median as True, lesser than as False
airbnb = airbnb.assign(
    is_expensive = airbnb.get('price') > 106.0
)
airbnb.head()

,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,is_expensive
0,Brooklyn,40.64749,-73.97237,Private room,149,1,9,365,True
1,Manhattan,40.75362,-73.98377,Entire home/apt,225,1,45,355,True
2,Manhattan,40.80902,-73.94190,Private room,150,3,0,365,True
3,Brooklyn,40.68514,-73.95976,Entire home/apt,89,1,270,194,False
4,Manhattan,40.79851,-73.94399,Entire home/apt,80,10,9,0,False


In [7]:
# split data into input features and labels
airbnb_df_X = airbnb.drop(columns=['is_expensive'])
airbnb_df_y = airbnb['is_expensive']

In [8]:
# one hot encoding for the categorical columns
airbnb_X_cat_col = ['neighbourhood_group', 'room_type']
airbnb_X = pd.get_dummies(columns=airbnb_X_cat_col, data=airbnb_df_X)

airbnb_y = airbnb_df_y.replace({True: 1, False: 0})

### Clean Olympic Dataset

The dataset is from https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results?select=athlete_events.csv, and includes historic data of all participants from the Olympic Games, from Athens 1896 to Rio 2016. There are 271116 unique values/observations. We want to see whether or not we can predict a gold medalist just by participant features.

In [10]:
# load the Olympic dataset
olympic = pd.read_csv('data/athlete_events.csv')
olympic.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [11]:
# These are the features we will be looking at for our classifier
olympic = olympic[['Sex', 'Age','Height','Weight','NOC','Sport','Medal']]
olympic.head()

,Sex,Age,Height,Weight,NOC,Sport,Medal
0,M,24.0,180.0,80.0,CHN,Basketball,NaN
1,M,23.0,170.0,60.0,CHN,Judo,NaN
2,M,24.0,NaN,NaN,DEN,Football,NaN
3,M,34.0,NaN,NaN,DEN,Tug-Of-War,Gold
4,F,21.0,185.0,82.0,NED,Speed Skating,NaN


In [12]:
# There are some NaN values in this dataset, so we want to remove those
olympic.shape

(271116, 7)

In [13]:
# This removes the NaN values from the numerical columns
olympic = olympic[olympic['Height'].notna()]
olympic = olympic[olympic['Weight'].notna()]
olympic = olympic[olympic['Age'].notna()]
olympic.head()

,Sex,Age,Height,Weight,NOC,Sport,Medal
0,M,24.0,180.0,80.0,CHN,Basketball,NaN
1,M,23.0,170.0,60.0,CHN,Judo,NaN
4,F,21.0,185.0,82.0,NED,Speed Skating,NaN
5,F,21.0,185.0,82.0,NED,Speed Skating,NaN
6,F,25.0,185.0,82.0,NED,Speed Skating,NaN


In [14]:
# After that cleaning, we are left with 206165 participants to work with
olympic.shape

(206165, 7)

In [15]:
# We want to make a column to see if the participant is a Gold Medalist
olympic['Gold'] = olympic['Medal']=='Gold'
olympic.head()

,Sex,Age,Height,Weight,NOC,Sport,Medal,Gold
0,M,24.0,180.0,80.0,CHN,Basketball,NaN,False
1,M,23.0,170.0,60.0,CHN,Judo,NaN,False
4,F,21.0,185.0,82.0,NED,Speed Skating,NaN,False
5,F,21.0,185.0,82.0,NED,Speed Skating,NaN,False
6,F,25.0,185.0,82.0,NED,Speed Skating,NaN,False


In [16]:
# We no longer need the Medal column
olympic = olympic.drop(columns=['Medal'])
olympic.head()

,Sex,Age,Height,Weight,NOC,Sport,Gold
0,M,24.0,180.0,80.0,CHN,Basketball,False
1,M,23.0,170.0,60.0,CHN,Judo,False
4,F,21.0,185.0,82.0,NED,Speed Skating,False
5,F,21.0,185.0,82.0,NED,Speed Skating,False
6,F,25.0,185.0,82.0,NED,Speed Skating,False


In [17]:
# split the olympic dataset into input features and labels
olympic_df_X = olympic.drop(columns=['Gold'])
olympic_df_y = olympic['Gold']

In [18]:
# one hot encode all the categorical columns
olympic_X_cat_col = ['Sex','NOC','Sport']
olympic_X = pd.get_dummies(columns=olympic_X_cat_col, data=olympic_df_X)
olympic_y = olympic_df_y.replace({True: 0, False: 1})

### Perform Trials

In [21]:
# Parameters for the model

tree_params = [
    {
        'max_depth': [2,3,4,5,7,10,13,15,18,None], 
        'min_samples_split':[2,3,5,7,10,15,20],
        'min_samples_leaf':[2,3,5,7,10,15,20]
    }
]

log_reg_params = [        
    {
        'solver': ['lbfgs'],
        'max_iter': [5000],
        'penalty': ['l2'],
        'C': 10**np.arange(-4, 5, 1, dtype='float32')
    },
    {
        'solver': ['saga'],
        'max_iter': [5000],
        'penalty': ['l1', 'l2'],
        'C': 10**np.arange(-4, 5, 1, dtype='float32')
    },
    {
        'solver': ['saga', 'lbfgs'],
        'max_iter': [5000],
        'penalty': ['none']
    }
]

perceptron_params = [
    {
        'penalty': ['l1', 'l2', 'elasticnet', 'none'],
        'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1]
    }
]

svc_params = [
    {
        'kernel': ['linear'],
        'C': 10 **np.array(np.arange(-3, 2, 2), dtype='float32')
    },
    {
        'kernel': ['poly'],
        'degree': [2, 3],
        'C': 10 **np.array(np.arange(-3, 2, 2), dtype='float32'),
    },
    {
        'kernel': ['rbf'],
        'C': 10 **np.array(np.arange(-3, 2, 2), dtype='float32'),
        'gamma': [0.001,0.01,0.1,1,2]
    }
]

knn_params = [
    {
        'n_neighbors': np.arange(1, 106, 4),
        'metric': ["euclidean", "manhattan", "minkowski"]
    }
]

forest_params = [
    {
        'n_estimators': [1024],
        'min_samples_split': [1, 2, 4, 6, 8, 12, 16, 20]
    }
]

# models that do not include SVM classifier
models_without_svm = {
    'tree': (DecisionTreeClassifier(), tree_params),
    'log_reg': (LogisticRegression(), log_reg_params),
    'perceptron': (Perceptron(), perceptron_params),
    'knn': (KNeighborsClassifier(), knn_params),
    'forest': (RandomForestClassifier(), forest_params)
}

# SVM model
models_only_svm = {
    'svm': (SVC(), svc_params)
}

In [23]:
# perform 7 trials using each of 6 algorithms on one dataset
def perform_trials(dataset_name, models, data_X, data_y):
    results_columns = ['dataset', 'model', 'trial',
                       'train_accuracy', 'train_precision',
                       'train_recall', 'train_specificity',
                       'train_f1', 'train_roc_auc', 
                       'train_log_loss', 'test_accuracy',
                       'test_precision', 'test_recall',
                       'test_specificity', 'test_f1',
                       'test_roc_auc', 'test_log_loss']
    scoring = {
        'accuracy': make_scorer(accuracy_score),
        'precision': make_scorer(precision_score),
        'recall': make_scorer(recall_score),
        'specificity': make_scorer(recall_score, pos_label=0),
        'f1': make_scorer(f1_score),
        'roc_auc': make_scorer(roc_auc_score),
        'log_loss': make_scorer(log_loss)
    }
    
    num_trials = 7
    
    data_results = pd.DataFrame(columns=results_columns)

    for model_name in models.keys():
        model = models[model_name][0]        
        model_params_grid = models[model_name][1]
        model_results = pd.DataFrame(columns=results_columns)
        
        # perform 7 trials using each model on the dataset
        for trial_count in range(num_trials):
            # pick 5000 samples with replacement to be in the training set
            X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, 
                                                                train_size=5000, 
                                                                random_state=trial_count)
            
            # grid search with 5 k-folds
            search = GridSearchCV(model, model_params_grid, cv=5, verbose=3,
                                  n_jobs=-1, refit=False, scoring=scoring)
            
            # fit grid search model with training set
            search.fit(X_train, y_train)
            
            # store 7 metrics calculated in one trial
            model_result = {
                'dataset': dataset_name,
                'model': model_name,
                'trial': trial_count + 1
            }
            
            for score_name in scoring.keys():
                # find the best parameters that make model achieves best score of the metric
                best_params = search.cv_results_['params'][np.argmin(search.cv_results_['rank_test_' + score_name])]
                # use best parameters to create the optimal model for the metric
                best_model = clone(model).set_params(**best_params)
                # train the optimal model
                best_model.fit(X_train, y_train)
                
                # compute metrics
                train_score = scoring[score_name](best_model, X_train, y_train)
                test_score = scoring[score_name](best_model, X_test, y_test)

                # append scores
                model_result['train_' + score_name] = train_score
                model_result['test_' + score_name] = test_score
            
            # append scores of one trial to the model_results dataframe
            model_results = model_results.append(model_result, ignore_index=True)
        
        # append model_results to data_results
        data_results = data_results.append(model_results, ignore_index=True)
        
        # store scores averaged over 7 trials
        avg_result = {
            'dataset': dataset_name,
            'model': model_name,
            'trial': 'avg',
            
            'train_accuracy': model_results.train_accuracy.mean(),
            'train_precision': model_results.train_precision.mean(),
            'train_recall': model_results.train_recall.mean(),
            'train_specificity': model_results.train_specificity.mean(),
            'train_f1': model_results.train_f1.mean(),
            'train_roc_auc': model_results.train_roc_auc.mean(),
            'train_log_loss': model_results.train_log_loss.mean(),
            
            'test_accuracy': model_results.test_accuracy.mean(),
            'test_precision': model_results.test_precision.mean(),
            'test_recall': model_results.test_recall.mean(),
            'test_specificity': model_results.test_specificity.mean(),
            'test_f1': model_results.test_f1.mean(),
            'test_roc_auc': model_results.test_roc_auc.mean(),
            'test_log_loss': model_results.test_log_loss.mean()
        }
        
        # append avg_result to the data_results dataframe
        data_results = data_results.append(avg_result, ignore_index=True)
    
    return data_results

### Chess Results

In [ ]:
chess_results_no_svm = perform_trials('chess', models_without_svm, chess_X, chess_y)
chess_results_no_svm.to_csv('results/chess_no_svm.csv')
chess_results_no_svm

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 490 candidates, totalling 2450 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:   46.1s
[Parallel(n_jobs=-1)]: Done 2256 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 2450 out of 2450 | elapsed:  1.2min finished


Fitting 5 folds for each of 490 candidates, totalling 2450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done 2256 tasks      | elapsed:   54.1s
[Parallel(n_jobs=-1)]: Done 2450 out of 2450 | elapsed:  1.0min finished


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:   41.2s
[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed:   55.6s
[Parallel(n_jobs=-1)]: Done 2450 out of 2450 | elapsed:  1.1min finished


Fitting 5 folds for each of 490 candidates, totalling 2450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   20.6s
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:   37.9s
[Parallel(n_jobs=-1)]: Done 2256 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 2450 out of 2450 | elapsed:  1.2min finished


Fitting 5 folds for each of 29 candidates, totalling 145 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.9s


In [ ]:
chess_results_svm = perform_trials('chess', models_only_svm, chess_X, chess_y)
chess_results_svm.to_csv('results/chess_svm.csv')
chess_results_svm

In [ ]:
chess_results_no_svm = pd.read_csv('results/chess_no_svm.csv')
chess_results_svm = pd.read_csv('results/chess_svm.csv')

In [ ]:
chess_results = chess_results_no_svm.append(chess_results_svm, ignore_index=True)
chess_results.to_csv('results/chess.csv')

### Shrooms Results

In [59]:
shrooms_results_no_svm = perform_trials('shrooms', models_without_svm, shrooms_X, shrooms_y)
shrooms_results_no_svm.to_csv('results/shrooms_no_svm.csv')
shrooms_results_no_svm

Fitting 5 folds for each of 490 candidates, totalling 2450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 368 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 1008 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 1904 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 2450 out of 2450 | elapsed:   16.6s finished


Fitting 5 folds for each of 490 candidates, totalling 2450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 368 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 1008 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 1904 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 2450 out of 2450 | elapsed:   16.5s finished


,dataset,model,trial,train_accuracy,train_precision,train_recall,train_specificity,train_f1,train_roc_auc,train_log_loss,test_accuracy,test_precision,test_recall,test_specificity,test_f1,test_roc_auc,test_log_loss
0,shrooms,tree,1,1.0000,1.0,1.000000,1.0,1.000000,1.000000,1.554272,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.625254
1,shrooms,tree,2,0.9994,1.0,0.998750,1.0,0.999375,0.999375,1.574997,0.998399,1.0,0.996702,1.0,0.998348,0.998351,1.592083
2,shrooms,tree,avg,0.9997,1.0,0.999375,1.0,0.999687,0.999688,1.564635,0.999200,1.0,0.998351,1.0,0.999174,0.999175,1.608669


In [ ]:
shrooms_results_svm = perform_trials('shrooms', models_only_svm, shrooms_X, shrooms_y)
shrooms_results_svm.to_csv('results/shrooms_svm.csv')
shrooms_results_svm

In [ ]:
shrooms_results_no_svm = pd.read_csv('results/shrooms_no_svm.csv')
shrooms_results_svm = pd.read_csv('results/shrooms_svm.csv')

In [ ]:
shrooms_results = shrooms_results_no_svm.append(shrooms_results_svm, ignore_index=True)
shrooms_results.to_csv('results/shrooms.csv')

### Results of Cardio Dataset

In [24]:
# running algorithms except SVM on cardio dataset
cardio_results_no_svm = perform_trials('cardio', models_without_svm, cardio_X, cardio_y)
cardio_results_no_svm

Fitting 5 folds for each of 490 candidates, totalling 2450 fits
Fitting 5 folds for each of 490 candidates, totalling 2450 fits
Fitting 5 folds for each of 490 candidates, totalling 2450 fits
Fitting 5 folds for each of 490 candidates, totalling 2450 fits
Fitting 5 folds for each of 490 candidates, totalling 2450 fits
Fitting 5 folds for each of 490 candidates, totalling 2450 fits
Fitting 5 folds for each of 490 candidates, totalling 2450 fits
Fitting 5 folds for each of 29 candidates, totalling 145 fits
Fitting 5 folds for each of 29 candidates, totalling 145 fits
Fitting 5 folds for each of 29 candidates, totalling 145 fits
Fitting 5 folds for each of 29 candidates, totalling 145 fits
Fitting 5 folds for each of 29 candidates, totalling 145 fits
Fitting 5 folds for each of 29 candidates, totalling 145 fits
Fitting 5 folds for each of 29 candidates, totalling 145 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100

C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [   nan 0.7122 0.7224 0.7244 0.729  0.7352 0.738  0.738 ]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.71018712 0.72262559 0.72602912 0.73219589 0.7414073
 0.7441335  0.74557375]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.7109301  0.71616149 0.71535747 0.71696713 0.71736711
 0.72058804 0.71776871]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.71342974 0.72853672 0.73330414 0.74085408 0.75277541
 0.75516031 0.75794045]
  category=UserWarning
C:\Use

Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [   nan 0.7072 0.7104 0.7134 0.714  0.7182 0.717  0.7176]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.71729122 0.72042873 0.72587774 0.72684657 0.73153264
 0.73224623 0.73332237]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.70368966 0.70643476 0.70447244 0.70408259 0.70722293
 0.70172734 0.70133749]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.71084999 0.71451931 0.72267343 0.72430359 0.72960389
 0.73286088 0.73449603]
  category=UserWarning
C:\Us

Fitting 5 folds for each of 8 candidates, totalling 40 fits


...

Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [   nan 0.7036 0.7082 0.7142 0.7196 0.728  0.7282 0.7314]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.70460275 0.70996288 0.71781816 0.7259626  0.73686675
 0.73774584 0.74218078]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.68095432 0.6842163  0.68707012 0.68747745 0.69237125
 0.69114843 0.69277692]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.72539004 0.73127162 0.74029354 0.75049501 0.76226203
 0.76383066 0.76853654]
  category=UserWarning
C:\Us

Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [   nan 0.7096 0.7186 0.724  0.7256 0.728  0.7286 0.7306]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.71396619 0.72523244 0.73327807 0.7361804  0.74195342
 0.74367672 0.74668793]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.70064192 0.70503713 0.70543553 0.70463872 0.70023872
 0.69864192 0.69904112]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.71857715 0.73218838 0.74259719 0.7466004  0.75580922
 0.75861082 0.76221082]
  category=UserWarning
C:\Us

,dataset,model,trial,train_accuracy,train_precision,train_recall,train_specificity,train_f1,train_roc_auc,train_log_loss,test_accuracy,test_precision,test_recall,test_specificity,test_f1,test_roc_auc,test_log_loss
0,cardio,tree,1,0.727200,0.769610,0.657407,0.814388,0.705400,0.726756,3.329554,0.718092,0.760048,0.651762,0.803292,0.698032,0.718082,12.502100
1,cardio,tree,2,0.730800,0.825455,0.771586,0.884584,0.736182,0.730678,3.377911,0.716569,0.823521,0.757578,0.888329,0.717287,0.716578,12.549928
2,cardio,tree,3,0.727600,0.773550,0.721763,0.817405,0.738914,0.728793,3.143042,0.723985,0.753861,0.691226,0.803083,0.711474,0.723870,12.463843
3,cardio,tree,4,0.737200,0.787893,0.746233,0.823245,0.724413,0.737665,3.426264,0.724569,0.761494,0.689836,0.797068,0.713895,0.724521,12.273081
4,cardio,tree,5,0.735000,0.797519,0.734182,0.807801,0.743664,0.735456,2.977256,0.726877,0.775533,0.706000,0.803805,0.711856,0.726788,12.826238
5,cardio,tree,6,0.734800,0.747525,0.731538,0.799922,0.720253,0.734063,3.315736,0.731446,0.761724,0.672682,0.804416,0.720963,0.731479,12.584985
6,cardio,tree,7,0.741400,0.784314,0.714229,0.819456,0.748012,0.741457,3.101595,0.722185,0.772991,0.658558,0.811395,0.695597,0.722577,12.450020
7,cardio,tree,avg,0.733429,0.783695,0.725277,0.823829,0.730977,0.733553,3.238766,0.723389,0.772739,0.689663,0.815913,0.709872,0.723414,12.521457
8,cardio,log_reg,1,0.729200,0.760739,0.663446,1.000000,0.708817,0.728782,17.158864,0.726646,0.757817,0.665702,1.000000,0.708874,0.726637,17.266731
9,cardio,log_reg,2,0.652000,0.661729,1.000000,0.658646,0.688273,0.652064,16.938208,0.647785,0.649235,1.000000,0.661212,0.681556,0.647767,17.306453


In [25]:
# running SVM algorithm on cardio dataset, generally take longer time to run than other algorithms combined
cardio_results_svm = perform_trials('cardio', models_only_svm, cardio_X, cardio_y)
cardio_results_svm

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits


,dataset,model,trial,train_accuracy,train_precision,train_recall,train_specificity,train_f1,train_roc_auc,train_log_loss,test_accuracy,test_precision,test_recall,test_specificity,test_f1,test_roc_auc,test_log_loss
0,cardio,svm,1,0.737200,0.805755,0.814412,1.000000,0.719471,0.736826,17.158864,0.727600,0.808387,0.800369,1.000000,0.710771,0.727591,17.266731
1,cardio,svm,2,0.733200,0.748567,1.000000,0.760604,0.732665,0.733509,16.931300,0.726415,0.733427,0.999784,0.757929,0.721300,0.726380,17.309641
2,cardio,svm,3,0.737000,0.785714,1.000000,0.824319,0.726668,0.737818,16.979656,0.727262,0.773072,0.999630,0.813709,0.716606,0.727192,17.304327
3,cardio,svm,4,0.735000,0.777726,1.000000,0.802260,0.741597,0.735625,17.110906,0.727154,0.752055,0.999969,0.786744,0.710793,0.727087,17.293700
4,cardio,svm,5,0.734400,0.770219,1.000000,0.805790,0.725506,0.734588,17.179985,0.725523,0.765986,1.000000,0.802022,0.715822,0.725488,17.287855
5,cardio,svm,6,0.728400,0.776371,0.821297,1.000000,0.704654,0.727103,16.924001,0.727862,0.784359,0.813484,1.000000,0.709144,0.727918,17.280547
6,cardio,svm,7,0.738600,0.903614,1.000000,0.996797,0.724668,0.738641,17.255972,0.728046,0.831545,1.000000,0.995173,0.712434,0.728008,17.282010
7,cardio,svm,avg,0.734829,0.795424,0.947959,0.884253,0.725033,0.734873,17.077241,0.727123,0.778404,0.944748,0.879368,0.713838,0.727095,17.289259


In [26]:
# combine results of svm and non-svm algorithms and save as a csv file
cardio_final_results = cardio_results_no_svm.append(cardio_results_svm, ignore_index=True)
cardio_final_results.to_csv('results/cardio_results.csv', index = False)

In [27]:
# display performance
pd.read_csv('results/cardio_results.csv')

,dataset,model,trial,train_accuracy,train_precision,train_recall,train_specificity,train_f1,train_roc_auc,train_log_loss,test_accuracy,test_precision,test_recall,test_specificity,test_f1,test_roc_auc,test_log_loss
0,cardio,tree,1,0.727200,0.769610,0.657407,0.814388,0.705400,0.726756,3.329554,0.718092,0.760048,0.651762,0.803292,0.698032,0.718082,12.502100
1,cardio,tree,2,0.730800,0.825455,0.771586,0.884584,0.736182,0.730678,3.377911,0.716569,0.823521,0.757578,0.888329,0.717287,0.716578,12.549928
2,cardio,tree,3,0.727600,0.773550,0.721763,0.817405,0.738914,0.728793,3.143042,0.723985,0.753861,0.691226,0.803083,0.711474,0.723870,12.463843
3,cardio,tree,4,0.737200,0.787893,0.746233,0.823245,0.724413,0.737665,3.426264,0.724569,0.761494,0.689836,0.797068,0.713895,0.724521,12.273081
4,cardio,tree,5,0.735000,0.797519,0.734182,0.807801,0.743664,0.735456,2.977256,0.726877,0.775533,0.706000,0.803805,0.711856,0.726788,12.826238
5,cardio,tree,6,0.734800,0.747525,0.731538,0.799922,0.720253,0.734063,3.315736,0.731446,0.761724,0.672682,0.804416,0.720963,0.731479,12.584985
6,cardio,tree,7,0.741400,0.784314,0.714229,0.819456,0.748012,0.741457,3.101595,0.722185,0.772991,0.658558,0.811395,0.695597,0.722577,12.450020
7,cardio,tree,avg,0.733429,0.783695,0.725277,0.823829,0.730977,0.733553,3.238766,0.723389,0.772739,0.689663,0.815913,0.709872,0.723414,12.521457
8,cardio,log_reg,1,0.729200,0.760739,0.663446,1.000000,0.708817,0.728782,17.158864,0.726646,0.757817,0.665702,1.000000,0.708874,0.726637,17.266731
9,cardio,log_reg,2,0.652000,0.661729,1.000000,0.658646,0.688273,0.652064,16.938208,0.647785,0.649235,1.000000,0.661212,0.681556,0.647767,17.306453


### Results of Australian Rain Dataset

In [21]:
# running algorithms except SVM on Australian rain dataset
aus_results_no_svm = perform_trials('aus', models_without_svm, aus_X, aus_y)
aus_results_no_svm

Fitting 5 folds for each of 490 candidates, totalling 2450 fits
Fitting 5 folds for each of 490 candidates, totalling 2450 fits
Fitting 5 folds for each of 490 candidates, totalling 2450 fits
Fitting 5 folds for each of 490 candidates, totalling 2450 fits
Fitting 5 folds for each of 490 candidates, totalling 2450 fits
Fitting 5 folds for each of 52 candidates, totalling 260 fits


C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [0.7874 0.7874    nan 0.8418 0.7874 0.7874    nan 0.8418 0.7874 0.7874
    nan 0.842  0.7874 0.8228    nan 0.8418 0.7874 0.8408    nan 0.8418
 0.8338 0.8414    nan 0.8418 0.8404 0.8418    nan 0.8418 0.8418 0.8416
    nan 0.8418 0.8418 0.8418    nan 0.8416 0.8418 0.8418    nan 0.8422
 0.8418 0.8418    nan 0.8418 0.8418 0.8418    nan 0.8418 0.842  0.8418
    nan 0.8418]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Fitting 5 folds for each of 52 candidates, totalling 260 fits


C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [0.78   0.78      nan 0.8418 0.78   0.78      nan 0.8412 0.78   0.78
    nan 0.8416 0.78   0.8304    nan 0.8412 0.78   0.8412    nan 0.8416
 0.8382 0.8414    nan 0.8416 0.8408 0.8414    nan 0.8414 0.8408 0.8414
    nan 0.8416 0.8412 0.8414    nan 0.8416 0.8416 0.8416    nan 0.8418
 0.8414 0.8416    nan 0.8416 0.8414 0.841     nan 0.8416 0.8416 0.8412
    nan 0.8412]
  category=UserWarning
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1323: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Fitting 5 folds for each of 52 candidates, totalling 260 fits


...

Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [   nan 0.8448 0.8462 0.8458 0.8458 0.8468 0.8462 0.8444]
  category=UserWarning


Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\Users\panyu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [   nan 0.8512 0.848  0.8496 0.85   0.848  0.8484 0.8462]
  category=UserWarning


,dataset,model,trial,train_accuracy,train_precision,train_recall,train_specificity,train_f1,train_auc,train_logloss,test_accuracy,test_precision,test_recall,test_specificity,test_f1,test_auc,test_logloss
0,aus,tree,1,0.84920,0.792060,0.394167,0.972060,0.526382,0.683114,5.208465e+00,0.832130,0.738826,0.363179,0.963920,0.486978,0.663550,5.798042
1,aus,tree,2,0.85380,0.760962,0.489091,0.956667,0.595462,0.722879,5.049596e+00,0.829467,0.670925,0.435163,0.940109,0.527917,0.687636,5.890023
2,aus,tree,3,0.84280,0.776718,0.378253,0.970183,0.508750,0.674218,5.429514e+00,0.829012,0.724368,0.355540,0.962000,0.476970,0.658770,5.905746
3,aus,tree,4,0.85380,0.745074,0.535872,0.946525,0.623390,0.741199,5.049602e+00,0.828855,0.656716,0.457168,0.933025,0.539068,0.695096,5.911174
4,aus,tree,5,0.86220,0.787349,0.524194,0.959320,0.629371,0.741757,4.759469e+00,0.830016,0.663578,0.453984,0.935460,0.539127,0.694722,5.871091
5,aus,tree,avg,0.85236,0.772432,0.464315,0.960951,0.576671,0.712633,5.099329e+00,0.829896,0.690883,0.413007,0.946903,0.514012,0.679955,5.875215
6,aus,log_reg,1,0.84360,0.727715,0.422389,0.957328,0.534524,0.689859,5.401891e+00,0.838167,0.742602,0.401473,0.960892,0.521180,0.681183,5.589523
7,aus,log_reg,2,0.84300,0.727931,0.457273,0.951795,0.561697,0.704534,5.422618e+00,0.839228,0.714293,0.443773,0.950193,0.547437,0.696983,5.552891
8,aus,log_reg,3,0.83620,0.712397,0.400558,0.955657,0.512790,0.678108,5.657479e+00,0.839784,0.730639,0.426674,0.955818,0.538739,0.691246,5.533707
9,aus,log_reg,4,0.83840,0.728959,0.452613,0.950917,0.558470,0.701765,5.581497e+00,0.839193,0.722869,0.430434,0.953752,0.539576,0.692093,5.554118


In [27]:
# running SVM algorithm on Australian rain dataset, generally take longer time to run than other algorithms combined
aus_results_svm = perform_trials('aus', models_only_svm, aus_X, aus_y)
aus_results_svm

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits


,dataset,model,trial,train_accuracy,train_precision,train_recall,train_specificity,train_f1,train_auc,train_logloss,test_accuracy,test_precision,test_recall,test_specificity,test_f1,test_auc,test_logloss
0,aus,svm,1,0.8552,0.746725,0.482596,0.955804,0.586286,0.719200,5.001243,0.843778,0.734472,0.450899,0.954189,0.558766,0.702544,5.395760
1,aus,svm,2,0.8562,0.750988,0.518182,0.951538,0.613233,0.734860,4.966706,0.843671,0.709647,0.484973,0.944321,0.576182,0.714647,5.399454
2,aus,svm,3,0.8512,0.744838,0.469331,0.955912,0.575827,0.712622,5.139398,0.842937,0.721490,0.462160,0.949890,0.563417,0.706025,5.424778
3,aus,svm,4,0.8480,0.729763,0.519043,0.943942,0.606625,0.731493,5.249929,0.840823,0.694537,0.487089,0.939961,0.572603,0.713525,5.497816
4,aus,svm,5,0.8544,0.758667,0.509857,0.953399,0.609861,0.731628,5.028875,0.844297,0.715122,0.480414,0.946335,0.574729,0.713374,5.377814
5,aus,svm,avg,0.8530,0.746196,0.499802,0.952119,0.598366,0.725960,5.077230,0.843101,0.715054,0.473107,0.946939,0.569139,0.710023,5.419124


In [28]:
# combine results of svm and non-svm algorithms and save as a csv file
aus_final_results = aus_results_no_svm.append(aus_results_svm, ignore_index=True)
aus_final_results.to_csv('results/aus_results.csv', index = False)

In [29]:
# display performance
pd.read_csv('results/aus_results.csv')

,dataset,model,trial,train_accuracy,train_precision,train_recall,train_specificity,train_f1,train_auc,train_logloss,test_accuracy,test_precision,test_recall,test_specificity,test_f1,test_auc,test_logloss
0,aus,tree,1,0.84920,0.792060,0.394167,0.972060,0.526382,0.683114,5.208465e+00,0.832130,0.738826,0.363179,0.963920,0.486978,0.663550,5.798042
1,aus,tree,2,0.85380,0.760962,0.489091,0.956667,0.595462,0.722879,5.049596e+00,0.829467,0.670925,0.435163,0.940109,0.527917,0.687636,5.890023
2,aus,tree,3,0.84280,0.776718,0.378253,0.970183,0.508750,0.674218,5.429514e+00,0.829012,0.724368,0.355540,0.962000,0.476970,0.658770,5.905746
3,aus,tree,4,0.85380,0.745074,0.535872,0.946525,0.623390,0.741199,5.049602e+00,0.828855,0.656716,0.457168,0.933025,0.539068,0.695096,5.911174
4,aus,tree,5,0.86220,0.787349,0.524194,0.959320,0.629371,0.741757,4.759469e+00,0.830016,0.663578,0.453984,0.935460,0.539127,0.694722,5.871091
5,aus,tree,avg,0.85236,0.772432,0.464315,0.960951,0.576671,0.712633,5.099329e+00,0.829896,0.690883,0.413007,0.946903,0.514012,0.679955,5.875215
6,aus,log_reg,1,0.84360,0.727715,0.422389,0.957328,0.534524,0.689859,5.401891e+00,0.838167,0.742602,0.401473,0.960892,0.521180,0.681183,5.589523
7,aus,log_reg,2,0.84300,0.727931,0.457273,0.951795,0.561697,0.704534,5.422618e+00,0.839228,0.714293,0.443773,0.950193,0.547437,0.696983,5.552891
8,aus,log_reg,3,0.83620,0.712397,0.400558,0.955657,0.512790,0.678108,5.657479e+00,0.839784,0.730639,0.426674,0.955818,0.538739,0.691246,5.533707
9,aus,log_reg,4,0.83840,0.728959,0.452613,0.950917,0.558470,0.701765,5.581497e+00,0.839193,0.722869,0.430434,0.953752,0.539576,0.692093,5.554118


### Results of AirBnB Price Dataset

In [22]:
# running algorithms except SVM on airbnb dataset
airbnb_results_no_svm = perform_trials('airbnb', models_without_svm, airbnb_X, airbnb_y)
airbnb_results_no_svm

Fitting 5 folds for each of 490 candidates, totalling 2450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 656 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 1936 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 2450 out of 2450 | elapsed:   15.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 490 candidates, totalling 2450 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 1528 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 2450 out of 2450 | elapsed:    9.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 490 candidates, totalling 2450 fits


...

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   55.5s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   32.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   55.5s finished


,dataset,model,trial,train_accuracy,train_precision,train_recall,train_specificity,train_f1,train_auc,train_logloss,test_accuracy,test_precision,test_recall,test_specificity,test_f1,test_auc,test_logloss
0,airbnb,tree,1,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
1,airbnb,tree,2,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
2,airbnb,tree,3,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
3,airbnb,tree,4,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
4,airbnb,tree,5,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
5,airbnb,tree,avg,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
6,airbnb,log_reg,1,0.98780,1.000000,0.975373,1.000000,0.987533,0.987687,4.213731e-01,0.988040,1.000000,0.976078,1.000000,0.987894,0.988039,4.130962e-01
7,airbnb,log_reg,2,0.99060,1.000000,0.981033,1.000000,0.990426,0.990517,3.246645e-01,0.987721,0.999580,0.975849,0.999590,0.987572,0.987719,4.241123e-01
8,airbnb,log_reg,3,0.98760,1.000000,0.974684,1.000000,0.987179,0.987342,4.282808e-01,0.989065,1.000000,0.978156,1.000000,0.988957,0.989078,3.776880e-01
9,airbnb,log_reg,4,0.98840,1.000000,0.976585,1.000000,0.988154,0.988292,4.006498e-01,0.989725,1.000000,0.979450,1.000000,0.989618,0.989725,3.548693e-01


In [23]:
airbnb_results_svm = perform_trials('airbnb', models_only_svm, airbnb_X, airbnb_y)
airbnb_results_svm

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  39 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   27.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   25.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 24 candidates, totalling 120 fits


...

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   25.7s finished


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   26.2s finished


,dataset,model,trial,train_accuracy,train_precision,train_recall,train_specificity,train_f1,train_auc,train_logloss,test_accuracy,test_precision,test_recall,test_specificity,test_f1,test_auc,test_logloss
0,airbnb,svm,1,1.00000,1.000000,1.0,1.000000,1.000000,1.000000,9.992007e-16,0.999795,0.999727,0.999863,0.999727,0.999795,0.999795,0.007082
1,airbnb,svm,2,1.00000,1.000000,1.0,1.000000,1.000000,1.000000,9.992007e-16,0.999704,0.999408,1.000000,0.999408,0.999704,0.999704,0.010229
2,airbnb,svm,3,0.99940,0.998777,1.0,0.998824,0.999388,0.999412,2.072375e-02,0.999271,0.998546,1.000000,0.998540,0.999272,0.999270,0.025180
3,airbnb,svm,4,1.00000,1.000000,1.0,1.000000,1.000000,1.000000,9.992007e-16,0.999863,0.999863,0.999863,0.999863,0.999863,0.999863,0.004721
4,airbnb,svm,5,1.00000,1.000000,1.0,1.000000,1.000000,1.000000,9.992007e-16,0.999954,1.000000,0.999909,1.000000,0.999954,0.999954,0.001574
5,airbnb,svm,avg,0.99988,0.999755,1.0,0.999765,0.999878,0.999882,4.144749e-03,0.999718,0.999509,0.999927,0.999508,0.999718,0.999717,0.009757


In [24]:
airbnb_final_results = airbnb_results_no_svm.append(airbnb_results_svm, ignore_index=True)
airbnb_final_results.to_csv('results/airbnb_results.csv', index = False)

In [25]:
# display performance
pd.read_csv('results/airbnb_results.csv')

,dataset,model,trial,train_accuracy,train_precision,train_recall,train_specificity,train_f1,train_auc,train_logloss,test_accuracy,test_precision,test_recall,test_specificity,test_f1,test_auc,test_logloss
0,airbnb,tree,1,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
1,airbnb,tree,2,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
2,airbnb,tree,3,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
3,airbnb,tree,4,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
4,airbnb,tree,5,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
5,airbnb,tree,avg,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
6,airbnb,log_reg,1,0.98780,1.000000,0.975373,1.000000,0.987533,0.987687,4.213731e-01,0.988040,1.000000,0.976078,1.000000,0.987894,0.988039,4.130962e-01
7,airbnb,log_reg,2,0.99060,1.000000,0.981033,1.000000,0.990426,0.990517,3.246645e-01,0.987721,0.999580,0.975849,0.999590,0.987572,0.987719,4.241123e-01
8,airbnb,log_reg,3,0.98760,1.000000,0.974684,1.000000,0.987179,0.987342,4.282808e-01,0.989065,1.000000,0.978156,1.000000,0.988957,0.989078,3.776880e-01
9,airbnb,log_reg,4,0.98840,1.000000,0.976585,1.000000,0.988154,0.988292,4.006498e-01,0.989725,1.000000,0.979450,1.000000,0.989618,0.989725,3.548693e-01


### Results of Olympic Gold Medal Dataset


In [26]:
# running algorithms except SVM on Olympic dataset
olympic_results_no_svm = perform_trials('olympic', models_without_svm, olympic_X, olympic_y)
olympic_results_no_svm

Fitting 5 folds for each of 490 candidates, totalling 2450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 440 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 1080 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 1892 tasks      | elapsed:   45.4s
[Parallel(n_jobs=-1)]: Done 2180 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 2450 out of 2450 | elapsed:  1.4min finished


Fitting 5 folds for each of 490 candidates, totalling 2450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 552 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 1000 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done 1576 tasks      | elapsed:   36.2s
[Parallel(n_jobs=-1)]: Done 2280 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 2450 out of 2450 | elapsed:  1.2min finished


Fitting 5 folds for each of 490 candidates, totalling 2450 fits


...

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.6min finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.6min finished


,dataset,model,trial,train_accuracy,train_precision,train_recall,train_specificity,train_f1,train_auc,train_logloss,test_accuracy,test_precision,test_recall,test_specificity,test_f1,test_auc,test_logloss
0,olympic,tree,1,0.95220,0.954655,0.997053,0.096386,0.975394,0.546719,1.650989,0.949827,0.953788,0.995456,0.069974,0.974177,0.532715,1.732942
1,olympic,tree,2,0.95680,0.958232,0.998108,0.144628,0.977764,0.571368,1.492108,0.950081,0.954340,0.995100,0.082620,0.974294,0.538860,1.724185
2,olympic,tree,3,0.95400,0.954911,0.998737,0.104000,0.976333,0.551368,1.588820,0.951000,0.953734,0.996816,0.067460,0.974799,0.532138,1.692422
3,olympic,tree,4,0.95000,0.952678,0.996839,0.074803,0.974259,0.535821,1.726976,0.948853,0.952527,0.995833,0.042469,0.973699,0.519151,1.766595
4,olympic,tree,5,0.96040,0.961701,0.998326,0.140271,0.979671,0.569299,1.367766,0.950568,0.953834,0.996214,0.072994,0.974563,0.534604,1.707359
5,olympic,tree,avg,0.95468,0.956436,0.997813,0.112018,0.976684,0.554915,1.565332,0.950066,0.953645,0.995884,0.067103,0.974306,0.531494,1.724701
6,olympic,log_reg,1,0.95020,0.950200,1.000000,0.000000,0.974464,0.500000,1.720071,0.950697,0.950697,1.000000,0.000000,0.974726,0.500000,1.702898
7,olympic,log_reg,2,0.95160,0.951600,1.000000,0.000000,0.975200,0.500000,1.671715,0.950662,0.950662,1.000000,0.000000,0.974707,0.500000,1.704100
8,olympic,log_reg,3,0.95000,0.950000,1.000000,0.000000,0.974359,0.500000,1.726979,0.950702,0.950702,1.000000,0.000000,0.974728,0.500000,1.702726
9,olympic,log_reg,4,0.94920,0.949200,1.000000,0.000000,0.973938,0.500000,1.754610,0.950722,0.950722,1.000000,0.000000,0.974739,0.500000,1.702040


In [27]:
olympic_results_svm = perform_trials('olympic', models_only_svm, olympic_X, olympic_y)
olympic_results_svm

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   17.4s
/Users/melchisedec/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  2.2min finished


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  2.1min finished


Fitting 5 folds for each of 24 candidates, totalling 120 fits


...

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  2.1min finished


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  2.0min finished


,dataset,model,trial,train_accuracy,train_precision,train_recall,train_specificity,train_f1,train_auc,train_logloss,test_accuracy,test_precision,test_recall,test_specificity,test_f1,test_auc,test_logloss
0,olympic,svm,1,0.95020,0.95020,1.0,0.0,0.974464,0.5,1.720071,0.950697,0.950697,1.0,0.0,0.974726,0.5,1.702898
1,olympic,svm,2,0.95160,0.95160,1.0,0.0,0.975200,0.5,1.671715,0.950662,0.950662,1.0,0.0,0.974707,0.5,1.704100
2,olympic,svm,3,0.95000,0.95000,1.0,0.0,0.974359,0.5,1.726979,0.950702,0.950702,1.0,0.0,0.974728,0.5,1.702726
3,olympic,svm,4,0.94920,0.94920,1.0,0.0,0.973938,0.5,1.754610,0.950722,0.950722,1.0,0.0,0.974739,0.5,1.702040
4,olympic,svm,5,0.95580,0.95580,1.0,0.0,0.977401,0.5,1.526649,0.950558,0.950558,1.0,0.0,0.974652,0.5,1.707706
5,olympic,svm,avg,0.95136,0.95136,1.0,0.0,0.975072,0.5,1.680005,0.950668,0.950668,1.0,0.0,0.974710,0.5,1.703894


In [28]:
olympic_final_results = olympic_results_no_svm.append(olympic_results_svm, ignore_index=True)
olympic_final_results.to_csv('results/olympic_results.csv', index = False)

In [29]:
# display performance
pd.read_csv('results/olympic_results.csv')

,dataset,model,trial,train_accuracy,train_precision,train_recall,train_specificity,train_f1,train_auc,train_logloss,test_accuracy,test_precision,test_recall,test_specificity,test_f1,test_auc,test_logloss
0,olympic,tree,1,0.95220,0.954655,0.997053,0.096386,0.975394,0.546719,1.650989,0.949827,0.953788,0.995456,0.069974,0.974177,0.532715,1.732942
1,olympic,tree,2,0.95680,0.958232,0.998108,0.144628,0.977764,0.571368,1.492108,0.950081,0.954340,0.995100,0.082620,0.974294,0.538860,1.724185
2,olympic,tree,3,0.95400,0.954911,0.998737,0.104000,0.976333,0.551368,1.588820,0.951000,0.953734,0.996816,0.067460,0.974799,0.532138,1.692422
3,olympic,tree,4,0.95000,0.952678,0.996839,0.074803,0.974259,0.535821,1.726976,0.948853,0.952527,0.995833,0.042469,0.973699,0.519151,1.766595
4,olympic,tree,5,0.96040,0.961701,0.998326,0.140271,0.979671,0.569299,1.367766,0.950568,0.953834,0.996214,0.072994,0.974563,0.534604,1.707359
5,olympic,tree,avg,0.95468,0.956436,0.997813,0.112018,0.976684,0.554915,1.565332,0.950066,0.953645,0.995884,0.067103,0.974306,0.531494,1.724701
6,olympic,log_reg,1,0.95020,0.950200,1.000000,0.000000,0.974464,0.500000,1.720071,0.950697,0.950697,1.000000,0.000000,0.974726,0.500000,1.702898
7,olympic,log_reg,2,0.95160,0.951600,1.000000,0.000000,0.975200,0.500000,1.671715,0.950662,0.950662,1.000000,0.000000,0.974707,0.500000,1.704100
8,olympic,log_reg,3,0.95000,0.950000,1.000000,0.000000,0.974359,0.500000,1.726979,0.950702,0.950702,1.000000,0.000000,0.974728,0.500000,1.702726
9,olympic,log_reg,4,0.94920,0.949200,1.000000,0.000000,0.973938,0.500000,1.754610,0.950722,0.950722,1.000000,0.000000,0.974739,0.500000,1.702040


In [ ]:
# combine datasets

In [ ]:
# create table 1

In [ ]:
# create table 2

In [ ]:
# create table 3